# Report

## Introduction and data

Diese Arbeit beantwortet die Forschungsfrage, wie gut sich das FDI-Rating professioneller Darts-Spieler:innen mit kombinierten Hard- und Soft-Facts prognostizieren lässt. Die Daten stammen aus dem Scraper (`pipeline.scraper`) für dartsorakel.com, werden via `pipeline.transform` aufbereitet und in PostgreSQL persistiert. Für den Report wird der konsolidierte Snapshot `data/processed/player_stats_all.csv` verwendet.

**Datenlage (aktuell):** ca. 2.9k Spieler:innen, 30+ Features, davon 2.4k mit beobachtetem `profile_fdi_rating`. Kernspalten: Rolling-12M Scoring (`last_12_months_*`), Checkout-Quoten, Legs-Win-% sowie Profilmerkmale (Alter, Tour-Card-Jahre, Preisgeld). Extrem schiefe Verteilungen bei Earnings → log-Transformation.

**OpenIntro-orientierte EDA (Kurzfassung):**
- `profile_fdi_rating` rechtsschief; Longtail aus Semi-Pros.
- Starke bivariate Kopplung: First-9 Average und Checkout-% korrelieren hoch mit FDI.
- Preisgeld wird nach Log-Transformation informativ; Rohwerte sind zu schief.
- Länder-Vergleich zeigt deutliche Streuungsunterschiede (ENG/NED breit, DACH enger).

**Datenqualität:** Fehlwerte v. a. bei `age` und `profile_tour_card_years`; für das Modell werden fehlende numerische Werte median-imputiert, kategoriale per Mode.

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
CSV_PATH = PROJECT_ROOT / "data/processed/player_stats_all.csv"

df = pd.read_csv(CSV_PATH)
target = "profile_fdi_rating"
rows, cols = df.shape
missing_target = df[target].isna().sum()
print(f"Rows: {rows:,} | Columns: {cols} | Missing target: {missing_target}")
print(df[[target, "last_12_months_averages", "last_12_months_first_9_averages", "log_total_earnings" if "log_total_earnings" in df else "profile_total_earnings"]]
      .describe(percentiles=[0.25, 0.5, 0.75, 0.95])
      .T)


## Methodology

Es wird ein voll automatisiertes `sklearn`-Pipeline-Setup trainiert:

- **Preprocessing:**
  - Entfernt Data-Leakage-Spalten (`api_rank`, `api_overall_stat`).
  - Numerisch: Median-Imputation + StandardScaler.
  - Kategorial: Mode-Imputation + OneHotEncoder (Country-Dummies, `handle_unknown="ignore"`).
  - Feature Engineering: `log_total_earnings`, `season_win_rate`, `checkout_combo`, `first9_delta`, `momentum_gap`, Quoten-Ratios (`first9_ratio`, `break_efficiency`, `power_scoring_ratio`), `tv_stage_delta`.

- **Modelle:**
  1) Linear Regression (Baseline, interpretierbar)
  2) Lasso (alpha=0.01, max_iter=40k) zur Shrinkage/Selektion
  3) Random Forest (GridSearch über n_estimators, max_depth, min_samples_leaf)

- **Validierung:** 80/20 Holdout + 5-fold Cross-Validation auf dem Train-Split; Metriken: MAE, RMSE, $R^2$. Zusätzlich TimeSeriesSplit als Robustness-Check (train nach Tour-Card/Alter sortiert).

- **Artefakte:** Bestes Modell wird als `models/best_fdi_pipeline.joblib` gespeichert; Metriken landen in `reports/metrics/latest_metrics.json` und werden von der Gradio-App gelesen.

## Results

Die jüngsten Trainingsläufe (siehe `reports/metrics/latest_metrics.json`) zeigen:

- **Lasso** liefert aktuell die beste Generalisierung: $R^2 \approx 0.929$, MAE ~35.3 FDI-Punkte, RMSE ~46.1.
- Lineare Regression liegt praktisch gleichauf; Random Forest bleibt leicht zurück (mehr Bias durch Parameter-Set).
- Feature-Importance (RF) und Koeffizienten (Lasso) bestätigen: First-9-Average, Checkout-%, Legs-Win-% und log Earnings treiben das FDI-Rating maßgeblich.

Im Test-Split sind die Fehlerbänder schmal; Residuen zeigen keine gravierenden systematischen Muster. Weitere Robustheitsprüfungen (Heteroskedastizität, Cook’s Distance) sind ergänzbar, siehe Notebook `fdi_rating_modeling.ipynb`.

In [ ]:
import json
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "reports").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
METRICS_PATH = PROJECT_ROOT / "reports/metrics/latest_metrics.json"

with open(METRICS_PATH, "r") as f:
    metrics_raw = json.load(f)["metrics"]

metrics_df = (
    pd.DataFrame(metrics_raw)
    .T
    [["mae", "rmse", "r2", "cv_mae_mean", "cv_mae_std"]]
    .rename_axis("model")
    .sort_values("r2", ascending=False)
)
metrics_df


## Discussion + Conclusion


**Schlussfolgerungen:**
- Hard-Scoring (First-9, 3-Dart-Average) und Checkout-Qualität erklären den Großteil der FDI-Varianz; log Earnings liefert zusätzlichen Langfrist-Signalanteil.
- Regularisierte lineare Modelle reichen aktuell aus; Baummodelle liefern keine Mehrleistung mit den verfügbaren Features.

**Limitationen:**
- Fehlwerte bei Alters-/Erfahrungsvariablen erhöhen Imputationsunsicherheit.
- Potenzielle Heteroskedastizität und Influential Points sind noch nicht formal ausgeschlossen (siehe geplante Diagnostik).
- Keine expliziten Zeitreihen-Features (Formkurven) im aktuellen Trainingsstand.

**Ausblick:**
- Form-/Momentum-Features als Rolling-Window-Trends ergänzen; weitere psychologische Proxies (Reiseaufwand, Back-to-Back Events).
- Formale Tests: Breusch-Pagan für Heteroskedastizität, Cook’s Distance für Influential Points in `fdi_rating_modeling.ipynb`.
- MLflow-Logging aktivieren und Modell-Monitoring (Drift) in der Scheduler-Pipeline verankern.


**Literatur & Quellen**
- Hardin, J. et al. (2022). *Introduction to Modern Statistics (2e).* OpenIntro Press.
- James, G. et al. (2023). *An Introduction to Statistical Learning with Applications in Python.* Springer.
- Lucas, A. (2023). "Predicting Darts Performance Using Momentum Metrics." *Journal of Sports Analytics*.
- Darts Orakel (https://dartsorakel.com) – Primärdatenquelle für FDI-Rating und Leistungsmetriken.
